In [1]:
import imageio
import h5py
import numpy as np

In [2]:
samples = ['s1','s2','s3']
p_noisy = ['0','10','15','20','25'] # # 0% is GT (clean) and the rest are noisy inputs
frame_count = 279 # number of frames per case

In [3]:
#loaded_data = None
for s in samples:
    loaded_sample = None
    for i in range(frame_count):
        loaded_frame = None
        for p in p_noisy:
            file_name = 'dataset/frames_1024/'+s+'_'+p+'_'+str(i+1).zfill(3)+'_1024x1024.jpg'
            #print(file_name)
            im = imageio.imread(file_name)
            np_im = np.array(im)
            #np_im = np.transpose(np_im,(2,0,1))
            np_im = np.expand_dims(np_im,axis = 0)
            #print(np_im.shape)
        if (loaded_frame is None):
            loaded_frame = np_im
        else:
            loaded_frame = np.append(loaded_frame,np_im,axis=0)
        #print(loaded_frame.shape)
        loaded_frame = np.expand_dims(loaded_frame,axis = 0)
        if (loaded_sample is None):
            loaded_sample = loaded_frame
        else:
            loaded_sample = np.append(loaded_sample,loaded_frame,axis=0)
    #print(loaded_sample.shape)
    loaded_sample = np.expand_dims(loaded_sample,axis = 0)
    hf = h5py.File('dataset/data_'+s+'_1024x1024.h5', 'w') # (frame,noise,width,height,channel)
    hf.create_dataset('images', data=loaded_sample) 
    hf.close()
#    if (loaded_data is None):
#        loaded_data = loaded_sample
#    else:
#        loaded_data = np.append(loaded_data,loaded_sample,axis=0)
#print(loaded_data.shape)
# (sample,frame,noise,width,height,channel)
#hf = h5py.File('dataset/alldata_1024x1024.h5', 'w')
#hf.create_dataset('images', data=loaded_data)
#hf.close()

FileNotFoundError: No such file: '/home/phacou/tomoGAN/TomoGAN/dataset/frames_1024/s1_0_001_1024x1024.jpg'

In [ ]:
# train and test data only based on samples s2 & s3
train_split = 0.8
train_data = None
test_data = None

for s in ['s2','s3']: # ignore s1 for now
    f = h5py.File("dataset/data_"+s+"_1024x1024.h5", "r")
    all_data = f['images']
    frame_range = np.arange(frame_count) # frame indices for all samples
    np.random.shuffle(frame_range)
    train_range = frame_range[:int(frame_count *  train_split)]
    train_range = np.sort(train_range)
    test_range = frame_range[int(frame_count *  train_split):]
    test_range = np.sort(test_range)
    print(all_data.shape)
    if (train_data is None):
        train_data = all_data[0,train_range,:,:,:,:]
    else:
        train_data = np.append(train_data,all_data[0,train_range,:,:,:,:],axis=0)
    if (test_data is None):
        test_data = all_data[0,test_range,:,:,:,:]
    else:
        test_data = np.append(test_data,all_data[0,test_range,:,:,:,:],axis=0)
    f.close()
train_data = np.transpose(train_data,(0,1,4,2,3)) # (frame,noise,channel,height,width)
test_data  = np.transpose(test_data,(0,1,4,2,3))

In [ ]:
data_train = {}
data_test = {}
for i,n in enumerate([0,10,15,20,25]): # n% noise, 0 for clean data, ground_truth
    tr_data = train_data[:,i,:,:,:]
    te_data = test_data[:,i,:,:,:]
    tr_data_flat = np.reshape(tr_data,(tr_data.shape[0]*tr_data.shape[1],tr_data.shape[2],tr_data.shape[3]))
    te_data_flat = np.reshape(te_data,(te_data.shape[0]*te_data.shape[1],te_data.shape[2],te_data.shape[3]))
    data_train[n]  =  tr_data_flat
    data_test[n] = te_data_flat

In [ ]:
hf = h5py.File('dataset/clean4train_s2_s3_80.h5', 'w')
hf.create_dataset('images', data=data_train[0])
hf.close()

hf = h5py.File('dataset/clean4test_s2_s3_20.h5', 'w')
hf.create_dataset('images', data=data_test[0])
hf.close()

for n in [10,15,20,25]:
    hf = h5py.File('dataset/noisy4train_n'+str(n)+'_s2_s3_80.h5', 'w')
    hf.create_dataset('images', data=data_train[n])
    hf.close()

    hf = h5py.File('dataset/noisy4test_n'+str(n)+'_s2_s3_20.h5', 'w')
    hf.create_dataset('images', data=data_test[n])
    hf.close()